In [1]:
import scipy.sparse
import json
import string
import pymorphy2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction.text import TfidfVectorizer

from tqdm import tqdm_notebook
from multiprocessing import Pool, cpu_count

In [2]:
def process_line(line):
    item = json.loads(line)
    del item['image'], item['content']
    item['title'] = item['title']
    return item

In [3]:
items_count = 0
with open('../data/interim/items.json') as items_file:
    for line in items_file:
        items_count += 1
items_count

328050

In [4]:
items = []
with open('../data/interim/items.json') as items_file:
    with Pool(cpu_count()) as p:
        items = list(tqdm_notebook(p.imap(process_line, items_file), total=items_count))
items = pd.DataFrame(items)
items.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


,itemId,title
0,0,Пять забавных «морщинистых» пород собак
1,1,История улицы Ирининской в Гомеле
2,2,Зачем Дудь всё время спрашивает гостей програм...
3,3,Светлана Зейналова крестила младшую дочь
4,4,"ГКНБ: бандиты в ГБАО делают вид, что рассталис..."


In [5]:
import gc
gc.collect()

19

In [6]:
data = []
row = []
col = []

train_lines = sum(1 for line in open('../data/interim/train.json','r'))

with open('../data/interim/train.json') as train_file:
    for i, line in enumerate(tqdm_notebook(train_file, total=train_lines)):
        json_line = json.loads(line)
        for item, rating in json_line['trainRatings'].items():
            data.append((-1) ** (int(rating) + 1))
            row.append(i)
            col.append(int(item))
        if i > 5:
            break
train = pd.DataFrame({'userId': row, 'itemId': col, 'rating': data})
del data, row, col
train.head()
# train_int = scipy.sparse.coo_matrix((data, (row, col)))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


,userId,itemId,rating
0,0,206495,-1
1,0,279694,-1
2,0,19718,-1
3,0,74707,-1
4,0,221548,-1


In [10]:
items = items.set_index('itemId')

In [13]:
items.iloc[train[(train.userId == 0) & (train.rating == 1)].itemId][10:20]

,title
itemId,
184990,В России собираются ввести налог на курение
75612,Отдых на халяву? Легко!
167043,"10 пар слов, значения которых мы всегда путаем"
204085,Тест с опущенными в воду руками на полминуты с...
236149,Что американцам кажется смешным. ТОП-5
306948,Недавно объявилась моя бывшая свекровь.
254293,"""Класть"" или ""ложить"": как правильно?"
154292,Прокрустовы туфельки
128617,"3 наречия, в которых делают ошибки безграмотны..."


In [15]:
items = pd.read_csv('../data/processed/processed_items.csv', index_col='itemId')

In [16]:
items['title'] = items['title'].str.split()
titles = list(items['title'].values)